In [51]:
#%pip install lxml

In [20]:
from lxml import etree
import configparser

config = configparser.RawConfigParser()
config.read('config.ini')
details_dict = dict(config.items('xpaths'))

parser = etree.XMLParser(encoding='utf-8')
docTree = etree.parse("pgs.jobxml", parser)

In [70]:
final_df = {}

for bigquery_column, xml_path in details_dict.items():
    target_element = docTree.xpath(xml_path)
    target_parent = target_element[0].getparent()
    final_df[bigquery_column] = target_parent.xpath('.//Content')[0].text.strip('"')

In [30]:
## xml path to extract
xml_path = 'job/proc/summary/Param/Path[text()="::title:append to log"]/../Content'
xml_path="//proc / summary/Param/Path[text()='::title:append to log']/../Content"
target_element = docTree.xpath(xml_path)
[i.text for i in target_element]

['"value1"', '"value2"']

In [44]:
from lxml import etree
parser = etree.XMLParser(encoding='utf-8')
docTree = etree.parse("pgs.jobxml", parser)

import pandas as pd
df = pd.read_csv("xmldata.csv")
resultant_json = {"proc": {"summary": {"title": {"text": [], "append_to_log": []}}}}

In [54]:
resultant_json = {"proc": {"summary": {"title": {"text": [], "append_to_log": []}}}}

def get_item(resultant_json, path, value):
    for item in path[:-1]:
        resultant_json = resultant_json[item]
    resultant_json[path[-1]].extend(value)
    #print(value)
    #print(resultant_json)

import re
for i, j in df.iterrows():
    path_prefix = j['File node']
    tag = j["File tag"]
    if not "@" in tag:
        xpath_constrcutor = ''
        xpath_constrcutor = "//" + "/".join(path_prefix.split(">")) + "/Param/Path[text()='{}']/../Content".format(tag)
        print(xpath_constrcutor)
        value = [element.text for element in docTree.xpath(xpath_constrcutor)]
        tag_list = []
        tag_list.extend(path_prefix.split(">"))
        tag_list.extend([re.sub("\s", "_", items.strip("::")) for items in re.split("(?<!:):(?!:)", tag)])
        tag_list = [element.strip() for element in tag_list]
        get_item(resultant_json, tag_list, value)
        

print(resultant_json)


//proc / summary/Param/Path[text()='::title:text']/../Content
//proc / summary/Param/Path[text()='::title:append to log']/../Content
{'proc': {'summary': {'title': {'text': [], 'append_to_log': ['"value1"', '"value2"']}}}}


In [39]:
import re
[re.sub("\s", "_", items.strip("::")) for items in re.split("(?<!:):(?!:)", "::title:append to log")]

['title', 'append_to_log']

In [51]:
q=[1,2,3]
q[-1]

3

In [17]:
a=''
a+"c"
a+"v"

'v'

In [60]:
final_df

{'proc.dataread.input.folder.dataset_name': 'h001j01_modelR999_vol_Leste_Epsilon_125x125x5_mrg_contour',
 'proc.dataread.file_key.name': 'subline'}

In [72]:
%pip install xmltodict

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [87]:
import xmltodict
with open("1.jobxml") as file:
    a = xmltodict.parse(file.read(), dict_constructor=dict)

In [90]:
import json
import json

with open("xmltojson.txt", 'w') as file:
    json_string = json.dumps(a, default=lambda o: o.__dict__, sort_keys=True, indent=2)
    file.write(json_string)

In [88]:
a

{'Parser': {'job': {'@name': 'h001j01_modelR999_Leste_125x125x5_PBR_DELTA',
   'info': {'Param': [{'Path': ':'},
     {'Path': '::number of tasks per slot', 'Content': '1'},
     {'Path': '::number of tasks'},
     {'Path': '::Release Version', 'Content': '"so-4.0.14"'},
     {'Path': '::SPArk Version', 'Content': '1.5'},
     {'Path': '::Release Home', 'Content': '"/pgs/spark/osprey/so-4.0.14"'},
     {'Path': '::Created By', 'Content': '"PandDa"'},
     {'Path': '::Site Name', 'Content': '""'},
     {'Path': '::Site PDB Server Name',
      'Content': '"dms.hou.compute.pgs.com"'},
     {'Path': '::Remote Site Name', 'Content': '""'},
     {'Path': '::Remote DMS Server Name', 'Content': '""'},
     {'Path': '::Simplified Template Name',
      'Content': '"h001j01_modelR999_Leste_125x125x5_PBR"'},
     {'Path': '::system type', 'Content': '"FLOW"'},
     {'Path': '::will share node', 'Content': '"YES"'},
     {'Path': '::application type', 'Content': '"FMPI"'},
     {'Path': '::processo

In [93]:
from google.cloud import bigquery
client = bigquery.Client()
bucket_name = 'gcp-bq-2021'

destination_uri = "gs://{}/{}".format(bucket_name, "sample.json")
dataset_ref = bigquery.DatasetReference("patents-public-data", "google_patents_research")
table_ref = dataset_ref.table("publications")
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    job_config=job_config,
    # Location must match that of the source table.
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

BadRequest: 400 Table dataset_reference {
  project_reference {
    project_id: "patents-public-data"
    gaia_id: 933024764304
  }
  dataset_id: "google_patents_research"
  dataset_uuid: "8a713a04-a63d-4273-983b-b4de0fad31f9"
}
table_id: "publications"
table_uuid: "5318fe0b-81d6-4d5f-bff4-8f90765513e6"
 too large to be exported to a single file. Specify a uri including a * to shard export. See 'Exporting data into one or more files' in https://cloud.google.com/bigquery/docs/exporting-data.